Ok, used my latinepi (which needs cleaning up) to download *a lot* of inscriptions from edh. Then used a combination of scripts & llm to annotate them. In this notebook, we clean out the unsuccessfully annotated inscriptions, leaving us with a core of real data, and then we generate a whole bunch of synthetic inscriptions (thank god for formulaic epigraphy, eh?) that are correctly annotated, and mix them both together. The goal is to train the latinCy spaCy model to recognize the elements of funerary inscriptions, for data extraction from transcriptions. Why not?

In [1]:
!mkdir assets         # To store your raw data files (jsonl, csv)
!mkdir configs        # To store configuration files
!mkdir scripts        # To store helper scripts (like data conversion)
!mkdir training       # To store the output of the training process
!mkdir corpus         # To store the processed .spacy files

In [ ]:
#!pip install -U spacy #already in colab
#!python -m spacy download en_core_web_lg
#!pip install "la-core-web-sm @ https://huggingface.co/latincy/la_core_web_sm/resolve/main/la_core_web_sm-any-py3-none-any.whl"
!pip install "la-core-web-lg @ https://huggingface.co/latincy/la_core_web_lg/resolve/main/la_core_web_lg-any-py3-none-any.whl"
#
# this is what we're going to retrain.
!pip install spacy-transformers

In [ ]:
# then you have to run this. It will say things have crashed. Ignore and continue.
import os
os.kill(os.getpid(), 9)

## clean up some real inscriptions to add with the synthetic ones

In [1]:
###### ok, try to clean up some real ones and then mix them in with the synthetic

## this is a couple hundred rows of real inscriptions that were annotated
## through combination of scripts & llm, but the results had issues with
## annotation offsets-->
!wget https://gist.githubusercontent.com/shawngraham/d949119d45f5cc661205a3bfbb266d86/raw/c93dfe9324d01478fe88f59f9cf9ee9d560dfc1e/annotations-to-clean-up-for-dataset.jsonl -O assets/to-clean.jsonl

# so, the 'output.jsonl' here is the result of the script at the bottom of the page that cleans up 753 rows of data from edh using this: https://github.com/shawngraham/latinepi/blob/main/edh-real-data-to-jsonl.py; the to-clean.jsonl is from another 1100 rows of data from edh (and there is a chance that perhaps there are duplicate rows, that's on my list of things to check/fix). Both were annotated using regex & dictionaries, but the first 753 made that a lot easier because I could use the column metadata for it (the second 1100 I obtained using my little command line thing latinepi for downloading from edh. Look, I'm making it all up as I go along.)-->
!wget https://gist.githubusercontent.com/shawngraham/8ed8e45daf57dc3b0425441d008005fd/raw/a5039a138e40ea1faf4485bec58518648266957f/edh-753.jsonl -O assets/output.jsonl

--2025-11-23 01:30:44--  https://gist.githubusercontent.com/shawngraham/d949119d45f5cc661205a3bfbb266d86/raw/c93dfe9324d01478fe88f59f9cf9ee9d560dfc1e/annotations-to-clean-up-for-dataset.jsonl
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 459000 (448K) [text/plain]
Saving to: ‘assets/to-clean.jsonl’

assets/to-clean.jso 100%[===================>] 448.24K  --.-KB/s    in 0.1s    

2025-11-23 01:30:45 (3.40 MB/s) - ‘assets/to-clean.jsonl’ saved [459000/459000]

--2025-11-23 01:30:45--  https://gist.githubusercontent.com/shawngraham/8ed8e45daf57dc3b0425441d008005fd/raw/a5039a138e40ea1faf4485bec58518648266957f/edh-753.jsonl
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting 

In [2]:

!cat assets/output.jsonl assets/to-clean.jsonl > assets/full_real_data.jsonl

In [3]:
## so we clean out the ones that are no good, just keeps the cleanest/easiest to fix
import json
import spacy

def clean_real_inscriptions(input_path, output_path, model='la_core_web_lg'):
    """
    Cleans real inscription data:
    1. Copies transcription → text
    2. Re-validates all annotation spans
    3. Drops records with errors or unalignable annotations
    4. Outputs clean subset ready for training
    """
    nlp = spacy.load(model)

    stats = {
        "total": 0,
        "has_error_flag": 0,
        "no_transcription": 0,
        "no_annotations": 0,
        "perfect_match": 0,
        "fixed_spans": 0,
        "dropped_records": 0,
        "saved": 0
    }

    salvaged = []

    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                record = json.loads(line)
                stats["total"] += 1
            except:
                continue

            # Skip flagged errors
            if record.get("_error"):
                stats["has_error_flag"] += 1
                continue

            # Use transcription as text
            text = record.get('transcription', '').strip()
            if not text:
                stats["no_transcription"] += 1
                continue

            # Update record to use text field
            record['text'] = text
            record.pop('transcription', None)  # Remove redundant field

            # Skip if no annotations
            annotations = record.get('annotations', [])
            if not isinstance(annotations, list) or not annotations:
                stats["no_annotations"] += 1
                continue

            # Validate/fix annotations
            doc = nlp.make_doc(text)
            validated_ents = []
            record_is_salvageable = True

            for entity in annotations:
                if not isinstance(entity, list) or len(entity) != 3:
                    continue

                start, end, label = entity

                # Validate span is within text bounds
                if start < 0 or end > len(text) or start >= end:
                    record_is_salvageable = False
                    break

                # Check if span matches actual text
                span_text = text[start:end]

                # Try alignment
                span = doc.char_span(start, end, label=label, alignment_mode="expand")

                if span is not None:
                    # Span aligned successfully
                    if span.start_char == start and span.end_char == end:
                        # Perfect match
                        validated_ents.append([start, end, label])
                        stats["perfect_match"] += 1
                    else:
                        # Adjusted but acceptable
                        validated_ents.append([span.start_char, span.end_char, label])
                        stats["fixed_spans"] += 1
                else:
                    # Could not align - record is bad
                    record_is_salvageable = False
                    break

            if record_is_salvageable and validated_ents:
                record['annotations'] = validated_ents
                salvaged.append(record)
                stats["saved"] += 1
            else:
                stats["dropped_records"] += 1

    # Save cleaned data
    with open(output_path, 'w', encoding='utf-8') as f:
        for record in salvaged:
            f.write(json.dumps(record) + '\n')

    print(f"\n✅ Real inscription cleaning complete")
    print(f"   Total records: {stats['total']}")
    print(f"   Flagged as error: {stats['has_error_flag']}")
    print(f"   No transcription: {stats['no_transcription']}")
    print(f"   No annotations: {stats['no_annotations']}")
    print(f"   Perfect annotations: {stats['perfect_match']}")
    print(f"   Fixed spans: {stats['fixed_spans']}")
    print(f"   Dropped (unrecoverable): {stats['dropped_records']}")
    print(f"   ✅ SAVED: {stats['saved']} clean records")
    print(f"   Saved to: {output_path}")

    return stats

# Run it
#clean_real_inscriptions('assets/to-clean.jsonl', 'assets/real_inscriptions_clean.jsonl')
clean_real_inscriptions('assets/full_real_data.jsonl', 'assets/real_inscriptions_clean.jsonl')


✅ Real inscription cleaning complete
   Total records: 1853
   Flagged as error: 221
   No transcription: 0
   No annotations: 25
   Perfect annotations: 5022
   Fixed spans: 8358
   Dropped (unrecoverable): 175
   ✅ SAVED: 1432 clean records
   Saved to: assets/real_inscriptions_clean.jsonl


{'total': 1853,
 'has_error_flag': 221,
 'no_transcription': 0,
 'no_annotations': 25,
 'perfect_match': 5022,
 'fixed_spans': 8358,
 'dropped_records': 175,
 'saved': 1432}

## make some synthetic

In [60]:
#!/usr/bin/env python3
"""
Advanced Synthetic Latin Inscription Generator (Final v4).
Features:
- Roman Numerals for Ages (Standard Epigraphic format).
- Correct Case Agreement (Nominative/Dative for Praenomina).
- 'Force Military' mode for targeted training data.
- Aggressive filtering of real data (removing ghost words, gender clashes).
- Smart inflection and abbreviation logic.
"""

import json
import random
import re
from collections import Counter

# --- LATIN UTILITIES ---

def to_roman(n):
    """Convert integer to Roman numeral."""
    if not isinstance(n, int) or n <= 0: return str(n)
    val = [
        100, 90, 50, 40,
        10, 9, 5, 4,
        1
    ]
    syb = [
        "C", "XC", "L", "XL",
        "X", "IX", "V", "IV",
        "I"
    ]
    roman_num = ''
    i = 0
    while  n > 0:
        for _ in range(n // val[i]):
            roman_num += syb[i]
            n -= val[i]
        i += 1
    return roman_num

# Map for common Praenomina (Nominative -> Dative)
PRAENOMEN_DAT_MAP = {
    'M.': 'M.', 'L.': 'L.', 'C.': 'C.', 'T.': 'T.', 'Q.': 'Q.', # Abbrevs don't change
    'Lucius': 'Lucio', 'Marcus': 'Marco', 'Quintus': 'Quinto',
    'Caius': 'Caio', 'Gaius': 'Gaio', 'Titus': 'Tito',
    'Publius': 'Publio', 'Sextus': 'Sexto', 'Aulus': 'Aulo',
    'Decimus': 'Decimo', 'Gnaeus': 'Gnaeo', 'Spurius': 'Spurio',
    'Tiberius': 'Tiberio', 'Manius': 'Manio', 'Servius': 'Servio',
    'Appius': 'Appio', 'Numerius': 'Numerio'
}

def inflect_praenomen(name, case='dative'):
    """Specific lookup for Praenomina case agreement."""
    if case == 'dative':
        return PRAENOMEN_DAT_MAP.get(name, name)
    return name

def inflect_latin(word, case='dative'):
    """
    Heuristic to decline Latin names.
    Prevents double-inflection and handles common suffixes.
    """
    word = word.strip()
    if not word or word.endswith('.'): return word

    if case == 'dative':
        # 2nd Declension (Marcus -> Marco)
        if word.endswith('us'): return word[:-2] + 'o'
        if word.endswith('ius'): return word[:-3] + 'io'

        # 1st Declension (Iulia -> Iuliae)
        if word.endswith('a'): return word + 'e'

        # 2nd Declension -er (Afer -> Afro, Niger -> Nigro)
        # Replaces the 'er' with 'ro' to avoid 'Celerro'
        if word.endswith('er'): return word[:-2] + 'ro'

        # 3rd Declension (Felix -> Felici)
        if word.endswith('ex'): return word[:-2] + 'ici'

        # Safety: If it already looks Dative/Genitive (ends in o or i), leave it.
        if word.endswith('o') or word.endswith('i'): return word

    return word

def abbreviate_text(word, p=0.7):
    """Randomly abbreviate common terms."""
    lookup = {
        'filius': 'f.', 'filia': 'f.', 'vixit': 'v.', 'annos': 'an.',
        'annis': 'an.', 'mensibus': 'm.', 'diebus': 'd.',
        'bene merenti': 'b. m.', 'hic situs est': 'h. s. e.',
        'miles': 'mil.', 'legio': 'leg.', 'frater': 'fr.',
        'coniunx': 'con.', 'posuit': 'p.', 'fecit': 'f.',
        'heres': 'h.', 'libertus': 'lib.', 'optio': 'opt.',
        'centurio': '7.', # Common epigraphic symbol
        'praefectus': 'praef.'
    }

    # 1. Check dictionary
    if word.lower() in lookup and random.random() < p:
        return lookup[word.lower()]

    # 2. Heuristic abbreviation (truncate at 3-4 chars)
    if len(word) > 4 and random.random() < 0.2:
        return word[:3] + '.'

    return word

# --- FREQUENCY EXTRACTOR ---

class FrequencyExtractor:
    @staticmethod
    def extract_frequencies(input_path):
        counters = {k: Counter() for k in ['PRAENOMEN', 'NOMEN', 'COGNOMEN', 'TRIBUS', 'ORIGO', 'OCCUPATION']}

        with open(input_path, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    data = json.loads(line)
                    text = data.get('text', '') or data.get('transcription', '')
                    for ann in data.get('annotations', []):
                        if len(ann) == 3:
                            s, e, lbl = ann
                            lbl = lbl.upper()
                            if lbl == "TRIBE": lbl = "TRIBUS"
                            if lbl == "ORIGIN": lbl = "ORIGO"

                            if lbl in counters:
                                snippet = text[s:e].strip()

                                # FIX 1: Handle CamelCase errors in OCR (ForoIuli -> Foro Iuli)
                                snippet = re.sub(r'([a-z])([A-Z])', r'\1 \2', snippet)

                                # Clean punctuation
                                clean_snip = re.sub(r'[^\w\.]', '', snippet)

                                # FIX 2: Filter garbage (< 3 chars), but allow abbrevs like "T."
                                if len(clean_snip) < 3 and not clean_snip.endswith('.'):
                                    continue

                                counters[lbl][clean_snip] += 1
                except: continue
        return counters

    @staticmethod
    def get_weighted_list(counter, top_n=50):
        if not counter: return []
        population = [word for word, count in counter.most_common(top_n)]
        weights = [count for word, count in counter.most_common(top_n)]
        return random.choices(population, weights=weights, k=1000)

# --- GENERATOR ---

class InscriptionGenerator:
    def __init__(self, data_pools=None):
        self.pools = data_pools if data_pools else {}

        # --- FIX 3: SANITIZE POOLS ---
        BAD_OCCUPATIONS = {'colonia', 'municipium', 'vixit', 'annos', 'fecit', 'patrono',
                           'filio', 'coniugi', 'fratri', 'sorori', 'gustin', 'flamin', 'uxor'}

        if self.pools:
            # Filter Names: Keep mostly Masculine Nominative (-us, -o, -x, -er)
            for key in ['NOMEN', 'COGNOMEN']:
                self.pools[key] = [
                    x for x in self.pools[key]
                    if len(x) > 3 and (x.endswith('us') or x.endswith('o') or x.endswith('x') or x.endswith('er') or x.endswith('i'))
                ]

            # Filter Occupations
            if 'OCCUPATION' in self.pools:
                self.pools['OCCUPATION'] = [
                    x for x in self.pools['OCCUPATION']
                    if len(x) > 3 and x.lower() not in BAD_OCCUPATIONS
                ]

        # --- DEFAULTS ---
        defaults = {
            'PRAENOMEN': ['M', 'L', 'C', 'T', 'Q', 'Sex', 'Ti', 'Cn', 'Sp', 'App', 'D'],
            'NOMEN': ['Iulius', 'Claudius', 'Flavius', 'Aelius', 'Ulpius', 'Valerius',
                      'Aurelius', 'Sempronius', 'Antonius', 'Domitius', 'Cassius', 'Pompeius'],
            'COGNOMEN': ['Victor', 'Felix', 'Maximus', 'Severus', 'Priscus', 'Fortunatus',
                         'Clemens', 'Primus', 'Secundus', 'Rufus', 'Saturninus', 'Marcellus'],
            'TRIBUS': ['Vel', 'Qui', 'Gal', 'Pap', 'Ser', 'Cla', 'Arn', 'Pol', 'Pup', 'Aem', 'Ani','Cam','Clu','Col','Cor','Cru','Esq','Fab','Fal','Hor','Lem','Mae','Men','Ouf','Pal','Pob','Pom','Pup','Qui','Rom','Sab','Sca','Ste','Sub','Ter','Tro','Vel','Vol'],
            'ORIGO': ['Roma', 'Hispania', 'Gallia', 'Dacia', 'Syria', 'Berytus', 'Carthago'],
            'OCCUPATION': ['miles', 'veteranus', 'medicus', 'scriba', 'negociator', 'faber']
        }

        for k, v in defaults.items():
            if k not in self.pools or not self.pools[k]:
                self.pools[k] = v

        # EXPANDED FORMULAS (Add Votive Sentences)
        self.FORMULAS = [
            'D M', 'D M', 'D M',
            'D M S', 'Dis Manibus', 'Dis Manibus Sacrum',
            'H S E', 'Hic Situs Est',
            'I O M', 'Iovi Optimo Maximo',
            'V S L M', 'Votum Solvit Libens Merito',
            'B M', 'Bene Merenti',
            'Ara Larum', 'Deo Herculi', 'Deo Silvano',
            'Pro Salute', 'In Honorem', 'Memoriae',
            'Posuit', 'Fecit', 'Dedit', 'Dedicavit'
        ]



        # EXPANDED MILITARY UNITS
        self.MILITARY = [
            # Legions
            'legio II Adiutrix', 'leg. II Adi.', 'leg II', 'legionis II',
            'legio X Gemina', 'leg. X Gem.', 'leg X',
            'legio XIII Gemina', 'leg. XIII',
            'legio VI Victrix', 'leg. VI Vic.',
            # Cohorts
            'cohors I', 'coh. I', 'coh I',
            'cohors II', 'coh. II',
            'cohors I Brittonum', 'coh. I Brit.',
            # Alae
            'ala II Asturum', 'ala II', 'alae II',
            # Numeri
            'numerus', 'num.', 'vexillatio'
        ]

        # Ranks triggering force logic
        self.RANKS = ['mil.', 'miles', 'veteranus', 'vet.', 'centurio', '7.',
                      'tribunus', 'trib. mil.', 'optio', 'praefectus', 'signifer']

    def _get(self, category):
        return random.choice(self.pools.get(category, ['UNKNOWN']))

    def _append(self, parts, spans, text, label=None):
        sep = " "
        # Randomize separators
        if random.random() < 0.15 and len(parts) > 0:
            sep = "\n"
        elif random.random() < 0.05 and len(parts) > 0:
            sep = " · "

        current_text = "".join(parts)
        if not current_text: sep = ""

        start_idx = len(current_text) + len(sep)
        parts.append(sep + text)

        if label:
            end_idx = start_idx + len(text)
            spans.append([start_idx, end_idx, label])

    def generate_funerary(self, force_military=False):
        parts = []
        spans = []

        # 1. Formula
        if random.random() < (0.5 if force_military else 0.9):
            form = random.choice(self.FORMULAS)
            if " " in form and form.islower() == False and random.random() < 0.4:
                form = "".join([w[0].upper() for w in form.split()])
            self._append(parts, spans, form, "FORMULA")

        # 2. Name Generation
        case = 'dative' if random.random() < 0.7 else 'nominative'
        prae = self._get('PRAENOMEN')
        nomen = self._get('NOMEN')
        cog = self._get('COGNOMEN')

        if case == 'dative':
            # FIX 4: Inflect Praenomen too
            prae = inflect_praenomen(prae, 'dative')
            nomen = inflect_latin(nomen, 'dative')
            cog = inflect_latin(cog, 'dative')

        self._append(parts, spans, prae, "PRAENOMEN")
        self._append(parts, spans, nomen, "NOMEN")

        # 3. Filiation
        if random.random() < 0.4:
            f_prae = random.choice(['M', 'L', 'C', 'T'])
            fil_str = f"{f_prae}. f."
            self._append(parts, spans, fil_str, "RELATIONSHIP")

        # 4. Tribe
        if random.random() < 0.5:
            self._append(parts, spans, self._get('TRIBUS'), "TRIBE")

        self._append(parts, spans, cog, "COGNOMEN")

        # 5. Origo
        if random.random() < 0.3:
            self._append(parts, spans, self._get('ORIGO'), "ORIGO")

        # 6. Occupation & Military Unit
        if force_military or random.random() < 0.3:

            if force_military:
                occ = random.choice(self.RANKS)
            else:
                occ = abbreviate_text(self._get('OCCUPATION'))

            self._append(parts, spans, occ, "OCCUPATION")

            # Logic: Is this a soldier?
            is_soldier = force_military or any(x in occ for x in ['mil', 'vet', 'cent', 'trib', 'praef', '7.'])

            if is_soldier:
                threshold = 1.0 if force_military else 0.8
                if random.random() < threshold:
                    unit = random.choice(self.MILITARY)
                    if '.' in unit and random.random() < 0.4:
                        unit = unit.replace('.', '')
                    self._append(parts, spans, unit, "MILITARY_UNIT")

        # 7. Age
        if random.random() < 0.8:
            years = random.randint(1, 90)
            # FEATURE: Convert to Roman Numerals (95% of time)
            if random.random() < 0.95:
                years_str = to_roman(years)
            else:
                years_str = str(years)

            variations = [
                f"vixit annis {years_str}",
                f"v. an. {years_str}",
                f"annorum {years_str}",
                f"ann. {years_str}",
                f"qui vixit ann. {years_str}"
            ]

            if random.random() < 0.4:
                age_str = f"annorum {years_str}"
            else:
                age_str = random.choice(variations)
            self._append(parts, spans, age_str, "AGE")

         # 8. Dedicator
        if random.random() < 0.6:
            ded_nomen = self._get('NOMEN')
            ded_cog = self._get('COGNOMEN')
            rel = abbreviate_text(random.choice(['coniunx', 'frater', 'heres', 'pater', 'filius']))

            self._append(parts, spans, ded_nomen, "NOMEN")
            self._append(parts, spans, ded_cog, "COGNOMEN")
            self._append(parts, spans, rel, "RELATIONSHIP")

            verb = random.choice(['fecit', 'posuit', 'faciendum curavit'])
            self._append(parts, spans, abbreviate_text(verb), "VERB")

        full_text = "".join(parts)
        return {"id": f"syn_{random.randint(100,999999)}", "text": full_text, "annotations": spans}

    def generate_peregrine(self):
        """
        Generates inscriptions for non-citizens (Peregrines).
        Structure: Cognomen + Filiation (Father's Name) + Age.
        Example: "Severus Tongini f. annorum XXI"
        """
        parts = []
        spans = []

        # 1. Formula (D M is common even for peregrines later on)
        if random.random() < 0.8:
            form = random.choice(self.FORMULAS)
            self._append(parts, spans, form, "FORMULA")

        # 2. Name: Single Name (Cognomen)
        # We pick from COGNOMEN pool but treat it as the primary identifier
        name = self._get('COGNOMEN')
        self._append(parts, spans, name, "COGNOMEN")

        # 3. Filiation: "Tongini f." (Father's name in Genitive + f.)
        if random.random() < 0.9: # High chance for peregrines
            father = self._get('COGNOMEN')
            # Heuristic: Convert Nominative to Genitive-ish
            # Severus -> Severi, Tonginus -> Tongini
            if father.endswith('us'): father_gen = father[:-2] + 'i'
            elif father.endswith('ius'): father_gen = father[:-3] + 'i'
            elif father.endswith('a'): father_gen = father + 'e'
            else: father_gen = father # Fallback

            rel_str = f"{father_gen} f."
            self._append(parts, spans, rel_str, "RELATIONSHIP")

        # 4. Age (CRITICAL: Use 'annorum' here to fix your Real Data gap)
        if random.random() < 0.9:
            years = random.randint(5, 80)
            years_str = to_roman(years)

            # Mix of 'vixit annis' and 'annorum' (Genitive Plural)
            variations = [
                f"annorum {years_str}",  # Matches 'Annorum XXI' in real data
                f"ann. {years_str}",
                f"an. {years_str}",
                f"vixit annis {years_str}"
            ]
            age_str = random.choice(variations)
            self._append(parts, spans, age_str, "AGE")

        # 5. Formula (Footer)
        if random.random() < 0.6:
            # H S E (Hic Situs Est) is very common for this demographic
            form = "H S E" if random.random() < 0.5 else "Hic Situs Est"
            if random.random() < 0.5:
                form += " S T T L" # Sit Tibi Terra Levis
            self._append(parts, spans, form, "FORMULA")

        full_text = "".join(parts)
        return {"id": f"syn_{random.randint(100,999999)}", "text": full_text, "annotations": spans}

if __name__ == "__main__":
    # 1. Load Stats
    try:
        print("Extracting stats from real data (assets/output.jsonl)...")
        # Make sure this file path points to your real data
        counts = FrequencyExtractor.extract_frequencies('assets/output.jsonl')
        pools = {k: FrequencyExtractor.get_weighted_list(v) for k, v in counts.items()}
        print("Stats loaded and filtered.")
    except Exception as e:
        print(f"Warning: Could not load real data ({e}). Using built-in defaults.")
        pools = None

    gen = InscriptionGenerator(pools)

    output_file = 'assets/synthetic_data.jsonl'

    with open(output_file, 'w', encoding='utf-8') as f:

        # Batch 1: General Population (Match Real Data Size ~1400)
        print("Generating  General Inscriptions...")
        for _ in range(600):
            entry = gen.generate_funerary(force_military=False)
            f.write(json.dumps(entry, ensure_ascii=False) + '\n')

        # Batch 2: Military Reinforcements (Boost Weak Class ~600)
        print("Generating  Military-Focused Inscriptions...")
        for _ in range(600):
            entry = gen.generate_funerary(force_military=True)
            f.write(json.dumps(entry, ensure_ascii=False) + '\n')

        # 3. Peregrine/Single Names (Fixes "Severus Tongini") - 600 records
        print("Generating  Peregrine Inscriptions...")
        for _ in range(1800):
            entry = gen.generate_peregrine() # <--- Calling the new method
            f.write(json.dumps(entry, ensure_ascii=False) + '\n')

    print(f"✅ Done. Saved records to {output_file}")

Extracting stats from real data (assets/output.jsonl)...
Stats loaded and filtered.
Generating  General Inscriptions...
Generating  Military-Focused Inscriptions...
Generating  Peregrine Inscriptions...
✅ Done. Saved records to assets/synthetic_data.jsonl


In [62]:
# STEP 5: Check sample output
with open('assets/synthetic_data.jsonl') as f:
    samples = [json.loads(f.readline()) for _ in range(10)]
    print("\nSample synthetic inscriptions:")
    for sample in samples:
        print(f"\n  {sample['text']}")
        labels = [ann[2] for ann in sample['annotations']]
        print(f"  Labels: {set(labels)}")


Sample synthetic inscriptions:

  AL Caio Mario
Galeria Mestrio annorum LXI Cornelius Optatus f.
faciendum curavit
  Labels: {'PRAENOMEN', 'RELATIONSHIP', 'TRIBE', 'VERB', 'COGNOMEN', 'AGE', 'NOMEN', 'FORMULA'}

  Dedicavit Lucio Valerio M. f. Pub Macro annorum XXVI Iulius Niger pater fac.
  Labels: {'PRAENOMEN', 'RELATIONSHIP', 'TRIBE', 'VERB', 'COGNOMEN', 'AGE', 'NOMEN', 'FORMULA'}

  In Honorem Marco Fabio Felix annorum LXXV Baebius Capito fr. p.
  Labels: {'PRAENOMEN', 'RELATIONSHIP', 'VERB', 'COGNOMEN', 'AGE', 'NOMEN', 'FORMULA'}

  H S E Lucio Aemilio Optato vixit annis XXXIX
  Labels: {'PRAENOMEN', 'COGNOMEN', 'AGE', 'NOMEN', 'FORMULA'}

  IOM Caius
Iulius · L. f. Verus mil.
Octavius Cassianus f.
fec.
  Labels: {'PRAENOMEN', 'RELATIONSHIP', 'VERB', 'OCCUPATION', 'COGNOMEN', 'NOMEN', 'FORMULA'}

  Marco Valerio
Avito
  Labels: {'NOMEN', 'COGNOMEN', 'PRAENOMEN'}

  HSE Quintus Attius C. f. Ani Maximus annorum LXXXI
  Labels: {'PRAENOMEN', 'RELATIONSHIP', 'TRIBE', 'COGNOMEN', 'AGE

## combine real & synthethic

In [63]:
# #!/usr/bin/env python3
#
"""
Harmonize real and synthetic data, resolve overlaps, and create stratified splits.
"""

import json
from sklearn.model_selection import train_test_split

# Centralized Mapping
# Ensure that keys cover both your Real Data CSV columns AND Synthetic Generator outputs
LABEL_MAPPING = {
    # --- INPUT (Left) : OUTPUT (Right) ---

    # Real Data uses 'PRAENOMEN', Syn uses 'PRAENOMEN' -> Target: PRAENOMEN
    "PRAENOMEN": "PRAENOMEN",
    "NOMEN": "NOMEN",
    "COGNOMEN": "COGNOMEN",

    # Real Data uses 'TRIBUS', Syn uses 'TRIBE' -> Target: TRIBE (or TRIBUS, pick one)
    "TRIBUS": "TRIBE",
    "TRIBE": "TRIBE",

    # Real Data uses 'ORIGO', Syn uses 'ORIGO' -> Target: ORIGO
    # (We map 'ORIGIN' just in case an old synthetic batch has it)
    "ORIGO": "ORIGO",
    "ORIGIN": "ORIGO",

    # Occupations
    "OCCUPATION": "OCCUPATION",
    "MILITARY_UNIT": "MILITARY_UNIT",

    # Formulas
    "FUNERARY_FORMULA": "FORMULA",
    "DEDICATORY_FORMULA": "FORMULA",
    "DEDICATION_TO_THE_GODS": "FORMULA",
    "FORMULA": "FORMULA",

    # The Fix for the 0.00 Score (Merge BENE_MERENTI into FORMULA)
    "BENE_MERENTI": "FORMULA",

    # Relations
    "RELATIONSHIP": "RELATIONSHIP",
    "FILIATION": "RELATIONSHIP",

    # Age
    "AGE": "AGE",
    "AGE_PHRASE": "AGE",

    # DROPPED LABELS (Noise reduction)
    "EPITHET": None,           # Too generic / confused with Cognomen
    "AGE_YEARS": None,
    "AGE_DAYS": None,
    "ACTION": None,
    "VERB": None,
    "SUPERNOMEN": "COGNOMEN"   # Treat supernomen as just another name
}

def resolve_overlaps(annotations):
    """
    Greedy overlap resolution:
    1. Sort by length (descending) -> Keep longest spans first.
    2. If lengths equal, sort by start position.
    """
    if not annotations:
        return []

    # Sort: Longest spans first, then by start position
    # x[1] - x[0] is length
    sorted_anns = sorted(annotations, key=lambda x: (x[1] - x[0], x[0]), reverse=True)

    kept = []
    # Create a set of occupied indices
    occupied = set()

    for start, end, label in sorted_anns:
        # Check if any character in this span is already occupied
        span_indices = set(range(start, end))
        if not span_indices & occupied:
            # No overlap, keep it
            kept.append([start, end, label])
            occupied.update(span_indices)

    # Sort back by start position for clean JSONL
    return sorted(kept, key=lambda x: x[0])


def harmonize_file(input_path, label_map):
    """
    Reads a JSONL, maps labels, removes overlaps, returns list of dicts.
    """
    cleaned_records = []
    dropped_anns = 0
    total_anns = 0

    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                record = json.loads(line)
            except:
                continue

            text = record.get('text', '') or record.get('transcription', '')
            # Ensure text isn't empty
            if not text:
                continue

            # Normalize annotations
            raw_anns = record.get('annotations', [])
            mapped_anns = []

            for entity in raw_anns:
                if not isinstance(entity, list) or len(entity) != 3:
                    continue

                s, e, original_label = entity

                # Check Bounds
                if s < 0 or e > len(text) or s >= e:
                    continue

                # Map Label
                # Use upper case to ensure case-insensitive matching
                target_label = label_map.get(original_label.upper())

                total_anns += 1
                if target_label:
                    mapped_anns.append([s, e, target_label])
                else:
                    dropped_anns += 1

            # Resolve Overlaps
            final_anns = resolve_overlaps(mapped_anns)

            # Construct cleaned record
            if final_anns:
                cleaned_records.append({
                    "text": text,
                    "annotations": final_anns,
                    "meta": {"source": input_path} # Useful for debugging
                })

    print(f"Processed {input_path}:")
    print(f"  - Kept Records: {len(cleaned_records)}")
    print(f"  - Dropped Annotations: {dropped_anns}/{total_anns}")

    return cleaned_records

def combine_and_split(real_path, synthetic_path, train_out, dev_out):
    print("=== HARMONIZING DATA ===")

    # 1. Process BOTH files through the same logic
    real_data = harmonize_file(real_path, LABEL_MAPPING)
    syn_data = harmonize_file(synthetic_path, LABEL_MAPPING)

    # 2. Add 'source_type' tag for stratification
    for r in real_data: r['stratify_tag'] = 'real'
    for r in syn_data: r['stratify_tag'] = 'synthetic'

    combined = real_data + syn_data
    stratify_labels = [r['stratify_tag'] for r in combined]

    print(f"\nTotal Combined: {len(combined)}")

    # 3. Stratified Split
    # This ensures your Dev set has exactly 20% Real and 20% Synthetic
    train, dev = train_test_split(
        combined,
        test_size=0.2,
        random_state=42,
        stratify=stratify_labels
    )

    # 4. Write Output (Removing temporary meta tags)
    def write_jsonl(data, filename):
        with open(filename, 'w', encoding='utf-8') as f:
            for record in data:
                # specific clean up before save
                out_record = {
                    "text": record['text'],
                    "annotations": record['annotations']
                }
                f.write(json.dumps(out_record, ensure_ascii=False) + '\n')

    write_jsonl(train, train_out)
    write_jsonl(dev, dev_out)

    print(f"\n✅ Split Complete (Stratified):")
    print(f"   Train: {len(train)}")
    print(f"   Dev:   {len(dev)}")
    print(f"   Files saved to {train_out} and {dev_out}")

if __name__ == "__main__":
    # Update paths to match your actual filenames
    combine_and_split(
        real_path='assets/real_inscriptions_clean.jsonl',  # From the first script
        synthetic_path='assets/synthetic_data.jsonl', # From the generator
        train_out='assets/train.jsonl',
        dev_out='assets/dev.jsonl'
    )


=== HARMONIZING DATA ===
Processed assets/real_inscriptions_clean.jsonl:
  - Kept Records: 1393
  - Dropped Annotations: 2292/11246
Processed assets/synthetic_data.jsonl:
  - Kept Records: 3000
  - Dropped Annotations: 705/18573

Total Combined: 4393

✅ Split Complete (Stratified):
   Train: 3514
   Dev:   879
   Files saved to assets/train.jsonl and assets/dev.jsonl


In [64]:
#!/usr/bin/env python3
"""
STEP 2: Clean text (remove Leiden brackets) and re-align annotations.
Fixes multi-word Praenomina.
"""
import json

def clean_text_and_spans(input_path, output_path):
    cleaned_records = []
    fixed_praenomina = 0

    print(f"Repairing {input_path}...")

    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                record = json.loads(line)
            except: continue

            original_text = record.get('text', '')
            original_anns = record.get('annotations', [])

            if not original_text: continue

            # --- STEP 1: CLEAN TEXT & BUILD MAP ---
            # Rebuild text char by char. mapping[old_index] = new_index
            new_text = ""
            mapping = []

            for char in original_text:
                mapping.append(len(new_text))

                # REMOVE CHARACTERS
                if char in ['/', '{', '}', '(', ')', '[', ']']:
                    # Replace '/' with space to prevent word merging
                    if char == '/':
                        new_text += " "
                    # Drop brackets entirely
                    else:
                        pass
                else:
                    new_text += char

            mapping.append(len(new_text)) # Map the end of string

            # --- STEP 2: REMAP ANNOTATIONS ---
            new_anns = []
            for start, end, label in original_anns:
                if start >= len(mapping) or end >= len(mapping):
                    continue

                new_start = mapping[start]
                new_end = mapping[end]

                if new_start == new_end: continue # Annotation collapsed

                span_text = new_text[new_start:new_end]

                # --- STEP 3: FIX BAD PRAENOMINA ---
                # Real data often labels "Titus Flavius" as one PRAENOMEN.
                # We fix this by shrinking the span to the first word only.
                if label == "PRAENOMEN" and " " in span_text.strip():
                    space_index = span_text.strip().find(" ")
                    if space_index > 0:
                        # Adjust end to the first space
                        # Note: We do this relative to the span text
                        actual_space_index = span_text.find(" ")
                        new_end = new_start + actual_space_index
                        fixed_praenomina += 1

                # Clean leading/trailing whitespace in span
                while new_end > new_start and new_text[new_end-1] == " ":
                    new_end -= 1
                while new_start < new_end and new_text[new_start] == " ":
                    new_start += 1

                if new_end > new_start:
                    new_anns.append([new_start, new_end, label])

            record['text'] = new_text
            record['annotations'] = new_anns
            cleaned_records.append(record)

    with open(output_path, 'w', encoding='utf-8') as f:
        for r in cleaned_records:
            f.write(json.dumps(r, ensure_ascii=False) + '\n')

    print(f"  - Fixed {fixed_praenomina} bad Praenomina.")
    print(f"  - Saved to {output_path}")

if __name__ == "__main__":
    clean_text_and_spans('assets/train.jsonl', 'assets/train_clean.jsonl')
    clean_text_and_spans('assets/dev.jsonl', 'assets/dev_clean.jsonl')

Repairing assets/train.jsonl...
  - Fixed 23 bad Praenomina.
  - Saved to assets/train_clean.jsonl
Repairing assets/dev.jsonl...
  - Fixed 7 bad Praenomina.
  - Saved to assets/dev_clean.jsonl


In [67]:
#!/usr/bin/env python3
"""
STEP 3: Align character indices to tokens (JSONL -> JSONL).
"""
import spacy
import json
from spacy.util import filter_spans

def align_annotations(input_path, output_path):
    # Load model (ensure this matches what you will train with later)
    try:
        nlp = spacy.load('la_core_web_lg')
    except OSError:
        print("Warning: 'la_core_web_lg' not found. Using blank 'la'.")
        nlp = spacy.blank("la")

    corrected_records = []
    stats = {
        "total": 0,
        "perfect": 0,
        "fixed": 0,
        "malformed": 0,
        "dropped_align": 0,
        "dropped_overlap": 0
    }

    print(f"Aligning {input_path}...")

    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                record = json.loads(line)
                stats["total"] += 1
            except:
                continue

            text = record.get('text', '')
            if not text:
                continue

            doc = nlp.make_doc(text)
            candidate_spans = []

            # 1. First Pass: Create Span objects
            for entity in record.get('annotations', []):
                if not isinstance(entity, list) or len(entity) != 3:
                    stats["malformed"] += 1
                    continue

                start, end, label = entity

                # Sanity check indices
                if start < 0 or end > len(text) or start >= end:
                    stats["malformed"] += 1
                    continue

                # Align
                span = doc.char_span(start, end, label=label, alignment_mode="expand")

                if span is None:
                    stats["dropped_align"] += 1
                else:
                    # Check if alignment changed the indices
                    if span.start_char == start and span.end_char == end:
                        stats["perfect"] += 1
                    else:
                        stats["fixed"] += 1

                    candidate_spans.append(span)

            # 2. Second Pass: Filter Overlaps (CRITICAL)
            # alignment_mode="expand" can introduce overlaps.
            # filter_spans keeps the longest span and discards conflicts.
            original_count = len(candidate_spans)
            final_spans = filter_spans(candidate_spans)

            if len(final_spans) < original_count:
                stats["dropped_overlap"] += (original_count - len(final_spans))

            # 3. Serialize back to JSON format
            clean_annotations = []
            for span in final_spans:
                clean_annotations.append([span.start_char, span.end_char, span.label_])

            record['annotations'] = clean_annotations
            corrected_records.append(record)

    with open(output_path, 'w', encoding='utf-8') as f:
        for record in corrected_records:
            f.write(json.dumps(record, ensure_ascii=False) + '\n')

    print(f"✅ Aligned {input_path} -> {output_path}")
    print(f"   Perfect Match:   {stats['perfect']}")
    print(f"   Realigned:       {stats['fixed']}")
    print(f"   Dropped (Align): {stats['dropped_align']} (No matching token boundary)")
    print(f"   Dropped (Over):  {stats['dropped_overlap']} (Created conflict)")

if __name__ == "__main__":
    align_annotations('assets/train_clean.jsonl', 'assets/train_focused_aligned.jsonl')
    align_annotations('assets/dev_clean.jsonl', 'assets/dev_focused_aligned.jsonl')

Aligning assets/train_clean.jsonl...
✅ Aligned assets/train_clean.jsonl -> assets/train_focused_aligned.jsonl
   Perfect Match:   18414
   Realigned:       1462
   Dropped (Align): 42 (No matching token boundary)
   Dropped (Over):  303 (Created conflict)
Aligning assets/dev_clean.jsonl...
✅ Aligned assets/dev_clean.jsonl -> assets/dev_focused_aligned.jsonl
   Perfect Match:   4767
   Realigned:       345
   Dropped (Align): 3 (No matching token boundary)
   Dropped (Over):  78 (Created conflict)


In [66]:
# STEP 4: Convert to .spacy format
from spacy.tokens import DocBin
from spacy.util import filter_spans

def create_spacy_file(input_path, output_path, model='la_core_web_lg'):
    nlp = spacy.load(model)
    db = DocBin()

    with open(input_path) as f:
        for line in f:
            record = json.loads(line)
            text = record.get('text', '')
            if not text:
                continue

            doc = nlp.make_doc(text)
            ents = []

            for start, end, label in record.get('annotations', []):
                span = doc.char_span(start, end, label=label, alignment_mode="expand")
                if span:
                    ents.append(span)

            doc.ents = filter_spans(ents)
            db.add(doc)

    db.to_disk(output_path)
    print(f"✅ Created {output_path}")

create_spacy_file('assets/train_focused_aligned.jsonl', 'corpus/train.spacy')
create_spacy_file('assets/dev_focused_aligned.jsonl', 'corpus/dev.spacy')

print("\n✅ Ready for training!")

✅ Created corpus/train.spacy
✅ Created corpus/dev.spacy

✅ Ready for training!


## Train the thing

In [48]:
import spacy
from pathlib import Path

# --- 1. Generate the base config ---
!python -m spacy init config configs/config.cfg --lang la --pipeline tok2vec,ner --optimize accuracy --force

print("✅ Base 'config.cfg' generated.")

# --- 2. Load and Modify ---
config_path = Path("configs/config.cfg")
config = spacy.util.load_config(config_path)

# Define the model we are using
LATIN_MODEL = "la_core_web_lg"

# --- Part A: Initialize Vectors (CRITICAL FOR LG MODELS) ---
# This loads the 300-dim vectors into the vocab so the tok2vec layer can find them.
config["initialize"]["vectors"] = LATIN_MODEL

# --- Part B: Source the tok2vec component ---
config["components"]["tok2vec"] = {
    "source": LATIN_MODEL,
    "component": "tok2vec"
}

# --- Part C: Connect NER to the vectors ---
# The tok2vec OUTPUT width is 96, even if the input vectors are 300.
config["components"]["ner"]["model"]["tok2vec"] = {
    "@architectures": "spacy.Tok2VecListener.v1",
    "width": 96,
    "upstream": "tok2vec"
}

config["nlp"]["batch_size"] = 200

# --- Part D: Paths and Freezing ---
config["paths"]["train"] = "./corpus/train.spacy"
config["paths"]["dev"] = "./corpus/dev.spacy"

# Freeze tok2vec so we don't ruin the pretrained Latin intelligence
config["training"]["frozen_components"] = ["tok2vec"]
# or unfreeze it, see what happens
#config["training"]["frozen_components"] = []
#config["training"]["max_epochs"]= 100
config["training"]["max_epochs"] = 30
# Mark it as annotating so it actually runs
config["training"]["annotating_components"] = ["tok2vec"]

# --- 3. Save ---
config.to_disk(config_path)

print(f"✅ Config updated for {LATIN_MODEL}. Listener width set to 96 (correct output dim).")

ℹ Generated config template specific for your use case
- Language: la
- Pipeline: ner
- Optimize for: accuracy
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
configs/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
✅ Base 'config.cfg' generated.
✅ Config updated for la_core_web_lg. Listener width set to 96 (correct output dim).


In [68]:
# Start the training process!
!python -m spacy train configs/config.cfg --output ./training/ #--gpu-id 0

ℹ Saving to output directory: training
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Frozen components: ['tok2vec']
ℹ Set annotations on update for: ['tok2vec']
ℹ Initial learn rate: 0.001
E    #       LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  --------  ------  ------  ------  ------
  0       0     68.19   11.26    7.77   20.44    0.11
  0     200   9154.33   64.95   71.81   59.30    0.65
  0     400   7610.89   74.07   78.44   70.16    0.74
  1     600   7532.65   75.43   78.03   73.00    0.75
  2     800   9725.54   77.11   82.42   72.45    0.77
  3    1000  10376.21   78.37   81.33   75.63    0.78
  4    1200  11351.72   79.56   86.24   73.84    0.80
  5    1400  15119.68   79.79   84.12   75.88    0.80
  7    1600  17112.00   80.01   83.63   76.70    0.80
  9    1800  19837.69   80.30   88.04

In [69]:
# CELL: Per-label evaluation
from spacy.training import Example
from spacy.util import filter_spans

nlp = spacy.load("training/model-best")

# Load dev.spacy
dev_docs = list(DocBin().from_disk("corpus/dev.spacy").get_docs(nlp.vocab))

# Score by label
from collections import defaultdict
scores = defaultdict(lambda: {"tp": 0, "fp": 0, "fn": 0})

for doc in dev_docs:
    gold_ents = {(e.start_char, e.end_char, e.label_) for e in doc.ents}
    pred_ents = {(e.start_char, e.end_char, e.label_) for e in nlp(doc.text).ents}

    for label in set([e[2] for e in gold_ents | pred_ents]):
        gold_with_label = {e for e in gold_ents if e[2] == label}
        pred_with_label = {e for e in pred_ents if e[2] == label}

        scores[label]["tp"] += len(gold_with_label & pred_with_label)
        scores[label]["fp"] += len(pred_with_label - gold_with_label)
        scores[label]["fn"] += len(gold_with_label - pred_with_label)

print("\nPer-label F1 on DEV set:")
print(f"{'LABEL':<25} {'PREC':<8} {'REC':<8} {'F1':<8}")
print("-" * 50)

for label in sorted(scores.keys()):
    tp = scores[label]["tp"]
    fp = scores[label]["fp"]
    fn = scores[label]["fn"]

    prec = tp / (tp + fp) if (tp + fp) > 0 else 0
    rec = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * prec * rec / (prec + rec) if (prec + rec) > 0 else 0

    print(f"{label:<25} {prec:.2f}     {rec:.2f}     {f1:.2f}")


Per-label F1 on DEV set:
LABEL                     PREC     REC      F1      
--------------------------------------------------
AGE                       0.96     0.96     0.96
COGNOMEN                  0.88     0.78     0.83
FORMULA                   0.94     0.68     0.79
MILITARY_UNIT             0.88     0.73     0.80
NOMEN                     0.75     0.76     0.75
OCCUPATION                0.84     0.63     0.72
ORIGO                     0.73     0.46     0.57
PRAENOMEN                 0.78     0.84     0.80
RELATIONSHIP              0.97     0.79     0.87
TRIBE                     0.75     0.73     0.74


In [70]:
## checking against the real/synth held-back data
# Load the trained model and dev data
import spacy
from spacy.tokens import DocBin

nlp = spacy.load("training/model-best")
dev_docs = list(DocBin().from_disk("corpus/dev.spacy").get_docs(nlp.vocab))

print(f"📊 Loaded {len(dev_docs)} dev inscriptions\n")

# Quick visual comparison: predictions vs gold labels
print("=" * 100)
print("SAMPLE PREDICTIONS vs GOLD LABELS")
print("=" * 100)

for i, gold_doc in enumerate(dev_docs[:10], 1):  # First 10 samples
    text = gold_doc.text
    pred_doc = nlp(text)

    print(f"\n[{i}] {text}")
    print(f"    {'ENTITY':<30} {'PREDICTED':<25} {'GOLD':<25} {'MATCH'}")
    print(f"    {'-'*30} {'-'*25} {'-'*25} {'-'*5}")

    # Collect all entity positions
    all_positions = set()
    pred_ents = {(e.start_char, e.end_char): (e.text, e.label_) for e in pred_doc.ents}
    gold_ents = {(e.start_char, e.end_char): (e.text, e.label_) for e in gold_doc.ents}
    all_positions.update(pred_ents.keys())
    all_positions.update(gold_ents.keys())

    # Compare
    for pos in sorted(all_positions):
        pred = pred_ents.get(pos, ('-', '-'))
        gold = gold_ents.get(pos, ('-', '-'))

        match = "✅" if pred == gold else "❌"
        entity_text = pred[0] if pred[0] != '-' else gold[0]

        print(f"    {entity_text:<30} {pred[1]:<25} {gold[1]:<25} {match}")

# Quick accuracy metrics
print("\n" + "=" * 100)
print("ACCURACY METRICS")
print("=" * 100)

total_pred = sum(len(nlp(doc.text).ents) for doc in dev_docs)
total_gold = sum(len(doc.ents) for doc in dev_docs)
correct = 0

for gold_doc in dev_docs:
    pred_doc = nlp(gold_doc.text)
    pred_set = {(e.start_char, e.end_char, e.label_) for e in pred_doc.ents}
    gold_set = {(e.start_char, e.end_char, e.label_) for e in gold_doc.ents}
    correct += len(pred_set & gold_set)

precision = correct / total_pred if total_pred > 0 else 0
recall = correct / total_gold if total_gold > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

print(f"\nTotal Predictions: {total_pred}")
print(f"Total Gold Labels: {total_gold}")
print(f"Correct (Exact Match): {correct}")
print(f"\n📈 Precision: {precision:.3f}")
print(f"📈 Recall:    {recall:.3f}")
print(f"📈 F1 Score:  {f1:.3f}")

📊 Loaded 879 dev inscriptions

SAMPLE PREDICTIONS vs GOLD LABELS

[1] Caius Lucretius Cai filius Papiria Iustus Opitergio miles legionis XV Apollinaris stipendiorum X annorum XXX heres de propio
    ENTITY                         PREDICTED                 GOLD                      MATCH
    ------------------------------ ------------------------- ------------------------- -----
    Caius                          PRAENOMEN                 PRAENOMEN                 ✅
    Lucretius                      NOMEN                     NOMEN                     ✅
    Papiria Iustus                 TRIBE                     TRIBE                     ✅

[2] D M S · Quintus Iulius Albanus faber · annorum XLVI
    ENTITY                         PREDICTED                 GOLD                      MATCH
    ------------------------------ ------------------------- ------------------------- -----
    D M S                          FORMULA                   FORMULA                   ✅
    Quintus         

## Ok, deploy the model on real data

In [71]:
import shutil
import spacy

# The best model should be saved as model-best during training
# Load it
nlp = spacy.load("training/model-best")

# Test on real inscriptions from your dataset
test_cases = [
    "Dis Manibus Lucio Ocratio Corrintho vixit annos XXX dies XI Ocratia Silvana filio piissimo bene merenti fecit",
    "Caius Pompeius Caius libertus librarius",
    "Dis Manibus Caius Pompeius vixit XXXXIII",
    "Dis Manibus Spediae Luci filiae Severae coniugi Luci Valeri Montani Quinti fili primi pili legionis XIII",
    "Figlinae Ocreana"
]

print("Testing on real inscriptions:\n")
for text in test_cases:
    doc = nlp(text)
    print(f"Text: {text[:70]}...")
    print("Entities predicted:")
    for ent in doc.ents:
        print(f"  {ent.text:25} → {ent.label_:15}")
    print()

Testing on real inscriptions:

Text: Dis Manibus Lucio Ocratio Corrintho vixit annos XXX dies XI Ocratia Si...
Entities predicted:
  Dis Manibus               → FORMULA        
  Lucio                     → NOMEN          
  Ocratio                   → NOMEN          
  Corrintho                 → COGNOMEN       
  filio                     → RELATIONSHIP   

Text: Caius Pompeius Caius libertus librarius...
Entities predicted:
  Caius                     → PRAENOMEN      

Text: Dis Manibus Caius Pompeius vixit XXXXIII...
Entities predicted:
  Dis Manibus               → FORMULA        
  Caius                     → PRAENOMEN      
  Pompeius                  → NOMEN          

Text: Dis Manibus Spediae Luci filiae Severae coniugi Luci Valeri Montani Qu...
Entities predicted:
  Dis Manibus               → FORMULA        
  Spediae                   → NOMEN          
  Severae                   → COGNOMEN       

Text: Figlinae Ocreana...
Entities predicted:



## Real Data Time

Loading in data from P.; should get some from RIB or somewhere.

In [75]:
import re

class LeidenProcessor:
    """Convert Leiden conventions to clean transcription (Corrected)"""

    # Common words that MUST be lowercase for the model to recognize them
    # This list covers relationships, age words, and formulas
    FORCE_LOWER = {
        'filius', 'filia', 'filii', 'filiae', 'f', 'fil',
        'vixit', 'v', 'annos', 'annis', 'annorum', 'ann', 'an',
        'mensis', 'mensibus', 'm', 'diebus', 'die', 'd',
        'hic', 'situs', 'est', 'sita', 'siti', 's', 'h', 'e',
        'sit', 'tibi', 'terra', 'levis', 't', 'l',
        'bene', 'merenti', 'posuit', 'fecit', 'faciendum', 'curavit',
        'pater', 'mater', 'frater', 'soror', 'uxor', 'maritus', 'coniunx',
        'de', 'suo', 'et', 'in', 'ex'
    }

    @staticmethod
    def process(leiden_text):
        # Step 1: Remove damage markers [3]
        text = re.sub(r'\[\d+\]', '', leiden_text)

        # Step 2: Remove question marks and simple brackets
        text = re.sub(r'\?', '', text)
        text = re.sub(r'\[([^\]]*)\]', r'\1', text)

        # Step 3: Join words broken across lines (Smart Join)
        # "Gem/ellian" -> "Gemellian"
        text = re.sub(r'([a-z])/([a-z])', r'\1\2', text, flags=re.IGNORECASE)
        text = re.sub(r'(\])/([a-z])', r'\1\2', text, flags=re.IGNORECASE) # Handle ]/letter

        # Step 4: Expand abbreviations (X(expansion) -> Xexpansion)
        def expand_abbrev(match):
            # If text is "Q(uintus)", we just want "Quintus"
            # If text is "an(norum)", we want "annorum"
            # We concatenate the parts.
            # Note: This assumes the Leiden text preserved the case logic.
            return match.group(1) + match.group(2)

        text = re.sub(r'([A-Za-z]+)\(([^)]*)\)', expand_abbrev, text)

        # Step 5: Replace remaining slashes with spaces (Line breaks)
        text = text.replace('/', ' ')
        text = text.replace('\\', ' ')

        # Step 6: Clean up whitespace and junk
        text = re.sub(r'[\[\]]', '', text)      # Remove remaining brackets
        text = re.sub(r'[<>]', '', text)        # Remove editorial corrections <A=B>
        text = re.sub(r'\s+', ' ', text).strip()

        # Step 7: INTELLIGENT CASING (The Fix)
        words = text.split()
        result = []

        for word in words:
            # Remove trailing punctuation for checking
            clean_word = word.rstrip('.,')

            if clean_word.lower() in LeidenProcessor.FORCE_LOWER:
                result.append(word.lower())
            else:
                # Preserve original casing from Leiden!
                # Editors usually capitalize names (Iulius) and lowercase verbs (fecit).
                # Trust the editor.
                result.append(word)

        return ' '.join(result)

In [79]:
import pandas as pd
import spacy
import re

# --- CONFIG ---
MODEL_PATH = "training/model-best"
INPUT_CSV = "assets/actual_inscriptions.csv"
OUTPUT_CSV = "final_annotated_inscriptions.csv"

# --- 1. LEIDEN PROCESSOR (The Lowercase Logic) ---
class LeidenProcessor:
    FORCE_LOWER = {
        'filius', 'filia', 'filii', 'filiae', 'f', 'fil',
        'vixit', 'v', 'annos', 'annis', 'annorum', 'ann', 'an',
        'mensis', 'mensibus', 'm', 'diebus', 'die', 'd',
        'hic', 'situs', 'est', 'sita', 'siti', 's', 'h', 'e',
        'sit', 'tibi', 'terra', 'levis', 't', 'l',
        'bene', 'merenti', 'posuit', 'fecit', 'faciendum', 'curavit',
        'pater', 'mater', 'frater', 'soror', 'uxor', 'maritus', 'coniunx',
        'de', 'suo', 'et', 'in', 'ex', 'sacrum', 'dis', 'manibus'
    }

    @staticmethod
    def process(leiden_text):
        if not isinstance(leiden_text, str): return ""
        text = re.sub(r'\[\d+\]', '', leiden_text) # Remove damage [3]
        text = re.sub(r'\?', '', text)
        text = re.sub(r'\[([^\]]*)\]', r'\1', text) # Flatten brackets
        text = re.sub(r'([a-z])/([a-z])', r'\1\2', text, flags=re.IGNORECASE) # Join lines

        # Expand abbreviations X(yz) -> Xyz
        text = re.sub(r'([A-Za-z]+)\(([^)]*)\)', lambda m: m.group(1)+m.group(2), text)

        text = text.replace('/', ' ').replace('\\', ' ')
        text = re.sub(r'[\[\]<>]', '', text)
        text = re.sub(r'\s+', ' ', text).strip()

        # Intelligent Casing
        words = text.split()
        result = []
        for word in words:
            clean = word.rstrip('.,')
            if clean.lower() in LeidenProcessor.FORCE_LOWER:
                result.append(word.lower())
            else:
                result.append(word) # Keep original casing for names
        return ' '.join(result)

# --- 2. LOAD RESOURCES ---
print("Loading model...")
nlp = spacy.load(MODEL_PATH)

# Regex to catch ages that might be lowercased (e.g., 'annorum l')
# Matches: annorum/vixit + space + (roman numerals OR digits)
age_regex = re.compile(r'(?i)\b(annorum|annis|ann|an|vixit|v)\W+(\d+|[IVXLCDM]+|[lxiv]+)\b')

# --- 3. PROCESSING LOOP ---
print(f"Processing {INPUT_CSV}...")
df = pd.read_csv(INPUT_CSV)
records = []

for idx, row in df.iterrows():
    raw_text = row.get('text', '')

    # A. Clean Text
    clean_text = LeidenProcessor.process(raw_text)
    if not clean_text: continue

    # B. AI Prediction
    doc = nlp(clean_text)

    preds = {}
    for ent in doc.ents:
        if ent.label_ not in preds: preds[ent.label_] = []
        preds[ent.label_].append(ent.text)

    # C. Regex Safety Net (Fixes the "lowercase l" issue)
    if 'AGE' not in preds:
        match = age_regex.search(clean_text)
        if match:
            preds['AGE'] = [match.group(0)]

    # D. Prepare Output
    row_out = {
        "id": row.get('id', f"row_{idx}"),
        "text_clean": clean_text,
        "text_raw": raw_text,
        "entity_count": len(doc.ents) + (1 if 'AGE' in preds and 'AGE' not in [e.label_ for e in doc.ents] else 0)
    }

    # Flatten entities to string
    labels_of_interest = [
        'PRAENOMEN', 'NOMEN', 'COGNOMEN', 'TRIBE', 'ORIGO',
        'FORMULA', 'OCCUPATION', 'MILITARY_UNIT', 'AGE', 'RELATIONSHIP'
    ]

    for label in labels_of_interest:
        row_out[label] = " | ".join(preds.get(label, []))

    records.append(row_out)

# --- 4. SAVE ---
out_df = pd.DataFrame(records)
out_df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')

print(f"✅ Success! Processed {len(records)} rows.")
print(out_df[['text_clean', 'PRAENOMEN', 'NOMEN', 'COGNOMEN', 'AGE']].head(10).to_string())

Loading model...
Processing assets/actual_inscriptions.csv...
✅ Success! Processed 138 rows.
                                                                                text_clean PRAENOMEN    NOMEN    COGNOMEN          AGE
0                   Iovi Optimo Maximo Quintus Cassius Cassianus animo libens votum posuit   Quintus  Cassius   Cassianus             
1      dis manibus sacrum Sycecale vixit anno mensibus v sorores Tricisma Salcea et Vegeta                                            
2                                                                       Figlina Gemelliana                                            
3  dis manibus sacrum Severus Tongini annorum XXI hic situs est sit tibi terra levis mater                        Severus  annorum XXI
4                                 Iovi Optimo Maximo Caius As Ms votum solvit libens animo     Caius                                  
5                     Turaesio Turcaudi filius et Pans Maeilonis filii sit eis terra levis       

In [77]:
!zip -r inscription_model.zip training/model-best

  adding: training/model-best/ (stored 0%)
  adding: training/model-best/meta.json (deflated 64%)
  adding: training/model-best/tok2vec/ (stored 0%)
  adding: training/model-best/tok2vec/cfg (stored 0%)
  adding: training/model-best/tok2vec/model (deflated 7%)
  adding: training/model-best/tokenizer (deflated 82%)
  adding: training/model-best/config.cfg (deflated 60%)
  adding: training/model-best/ner/ (stored 0%)
  adding: training/model-best/ner/cfg (deflated 33%)
  adding: training/model-best/ner/moves (deflated 72%)
  adding: training/model-best/ner/model (deflated 7%)
  adding: training/model-best/vocab/ (stored 0%)
  adding: training/model-best/vocab/lookups.bin (stored 0%)
  adding: training/model-best/vocab/strings.json (deflated 82%)
  adding: training/model-best/vocab/key2row (stored 0%)
  adding: training/model-best/vocab/vectors (deflated 10%)
  adding: training/model-best/vocab/vectors.cfg (deflated 28%)


## And Now We Try The LATINEPI Thing

Which was initially supposed to be a ML-free approach to extracting structured data. It uses a whole bunch of pattern matching, but now also will use the trained underlying latinCy model we built above _first_ then the other stuff. Worth a shot... (and I should put some more effort into the patterns maybe).

In [54]:
# Clone fresh
!git clone https://github.com/shawngraham/latinepi.git
%cd latinepi
!pip install -e .
%cd ..


Cloning into 'latinepi'...
remote: Enumerating objects: 389, done.
remote: Total 389 (delta 0), reused 0 (delta 0), pack-reused 389 (from 1)
Receiving objects: 100% (389/389), 941.91 KiB | 2.03 MiB/s, done.
Resolving deltas: 100% (200/200), done.
/content/latinepi
Obtaining file:///content/latinepi
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for latinepi (pyproject.toml) ... done
  Created wheel for latinepi: filename=latinepi-0.1.0-0.editable-py3-none-any.whl size=10358 sha256=0613d9e152c40a841b66ce470616a508d84c23eaab188c5253c24df8e21e20d7
  Stored in directory: /tmp/pip-ephem-wheel-cache-rydskdk9/wheels/27/49/bd/699880d166b00e5fc8881fe68f1b9364f78c0a2dde49d64999
Successfully built latinepi
/content


In [55]:
!latinepi --input assets/actual_inscriptions.csv --output results.json --use-ml-hybrid

/content/latinepi/latinepi/grammar_patterns.py:291: SyntaxWarning: invalid escape sequence '\s'
  'BENE\s+MERENTI': ('well-deserving', 0.75),
Using ML+regex hybrid extractor with:
  - Fine-tuned latinCy spaCy model (primary)
  - Regex pattern validation (confidence boosting)
Processing 138 inscription(s)...
✅ Loaded latinCy model from training/model-best
Processed inscription 1/138
✅ Loaded latinCy model from training/model-best
Processed inscription 2/138
✅ Loaded latinCy model from training/model-best
Processed inscription 3/138
✅ Loaded latinCy model from training/model-best
Processed inscription 4/138
✅ Loaded latinCy model from training/model-best
Processed inscription 5/138
✅ Loaded latinCy model from training/model-best
Processed inscription 6/138
✅ Loaded latinCy model from training/model-best
Processed inscription 7/138
✅ Loaded latinCy model from training/model-best
Processed inscription 8/138
✅ Loaded latinCy model from training/model-best
Processed inscription 9/138
✅ Loade

In [56]:
import sys
sys.path.insert(0, '/content/latinepi')

# Import directly from the module file
from latinepi.hybrid_ml_extractor import extract_entities_hybrid as extract_entities_ml_hybrid

text = "D(is) M(anibus) // P(ublius) Ael(ius) P(ubli) f(ilius) Maecia / Mestrius Pela(gonia) / opt(io) leg(ionis) II ad(iutricis) / |(centuria) Attei Dextri / an(norum) XXXVIIII stip(endiorum) / XVIIII h(ic) s(itus) e(st) opti/ones leg(ionis) eiusde(m) / f(aciendum) [c(uraverunt)]"
entities = extract_entities_ml_hybrid(text)
print(entities)

✅ Loaded latinCy model from training/model-best
{'cognomen': {'value': 'Mestrius', 'confidence': 0.85}, 'years_lived': {'value': '39', 'confidence': 0.85}, 'military_service': {'value': 'Miles', 'confidence': 0.75}, 'tribe': {'value': 'Maecia', 'confidence': 0.88}}


In [ ]:
!latinepi --input assets/actual_inscriptions.csv --output results.json --use-ml-hybrid

# the end.

In [58]:
## some debugging
import json
from collections import Counter

def check_label_counts(path):
    c = Counter()
    with open(path) as f:
        for line in f:
            rec = json.loads(line)
            for ent in rec['annotations']:
                c[ent[2]] += 1
    print(f"Stats for {path}:")
    print(c.most_common())

check_label_counts('assets/train.jsonl')
check_label_counts('assets/dev.jsonl')

Stats for assets/train.jsonl:
[('COGNOMEN', 3750), ('NOMEN', 3297), ('FORMULA', 2942), ('RELATIONSHIP', 2397), ('PRAENOMEN', 2217), ('AGE', 1615), ('OCCUPATION', 1171), ('TRIBE', 851), ('MILITARY_UNIT', 680), ('ORIGO', 473)]
Stats for assets/dev.jsonl:
[('COGNOMEN', 939), ('NOMEN', 841), ('FORMULA', 812), ('RELATIONSHIP', 622), ('PRAENOMEN', 560), ('AGE', 395), ('OCCUPATION', 281), ('TRIBE', 186), ('MILITARY_UNIT', 173), ('ORIGO', 105)]
